In [1]:
# Jupyter notebook for testing deepModel library
# author: Fabrizio Romanelli
# email : fabrizio.romanelli@gmail.com
# date  : 04/10/2023

# Import the deepModel library
import deepModel as dm

# Initialize the environment
dm.initialize(CPU=20, GPU=1, VERBOSE='2', NPARRAYS=True)

2023-10-09 17:11:36.032335: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-09 17:11:36.142581: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-09 17:11:37.839136: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-09 17:11:37.843036: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open f

In [3]:
# Multivariate DNN dev test
mDnn = dm.multivariateDNN(name="multivariate DNN test", inputN=2)
mDnn.setInputs([{'shape': (2,), 'name': 'gino'}, {'shape': (2,), 'name': 'pino'}])

innerLayers = [[{'units': 16, 'activation': 'elu'}, {'units': 16, 'activation': 'elu'}, {'units': 16, 'activation': 'elu'}, {'units': 3, 'activation': 'linear'}]]
innerLayers.append([{'units': 16, 'activation': 'elu'}, {'units': 16, 'activation': 'elu'}, {'units': 16, 'activation': 'elu'}, {'units': 3, 'activation': 'linear'}])
innerLayers.append([{'units': 16, 'activation': 'elu'}, {'units': 16, 'activation': 'elu'}, {'units': 16, 'activation': 'elu'}, {'units': 3, 'activation': 'linear'}])

mDnn.setLayers(innerLayers)

outputLayers = [{'units': 32, 'activation': 'elu'}, {'units': 64, 'activation': 'elu'}, {'units': 128, 'activation': 'elu'}, {'units': 64, 'activation': 'elu'}, {'units': 32, 'activation': 'elu'}, {'units': 1, 'activation': 'linear'}]

mDnn.setOutLayers(outputLayers)
mDnn.setModelConfiguration(optimizer='adam', loss='mse')

mDnn.build()
# mDnn.summary()

import numpy as np

x1 = np.array([0,0,0,0,0,0])
x2 = np.array([1,1,1,1,1,1])
X1 = np.array([x1,x2]).T
x3 = np.array([0,0,0,0,0,0])
x4 = np.array([1,1,1,1,1,1])
X2 = np.array([x3,x4]).T
y  = np.array([0,1,0,1,0,1])

mDnn.fit(x=[X1,X2], y=y, epochs=20, shuffle=True, verbose=1)

[DM] Building model...
[DM] Model built!


TypeError: DNN.fit() takes 1 positional argument but 3 were given

In [ ]:
cnn = dm.CNN(2,'test')
cnn.getModel()
cnn.summary()

# # Make an instance of the RNN
# _RNN = dm.RNN(_layers=3,_name="test 1")

# print(_RNN.getLayers())
# print(_RNN.getName())

# # Make an instance of the CNN
# _CNN = dm.CNN(_layers=13,_name="test 2")

# print(_CNN.getLayers())
# print(_CNN.getName())